In [1]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load data
usersorder_df = pd.read_excel(r"USER AND RESTRAUNT.xlsx")  # Contains columns: user_id, rest_id, cost, rating, location
restaurants_df = pd.read_csv(r"BangaloreZomatoData_with_rest_id.csv")


In [2]:
usersorder_df .head()

,rest_id,Name,user_id,cost,rating,location,Cuisines
0,R6892,Indian Food Court,U0350,1000,4,"Electronic City, Bangalore","North Indian, Chinese, Fast Food"
1,R7304,Ludos Pizza,U0825,800,2,"JP Nagar, Bangalore","Pizza, Desserts"
2,R1631,Seth Chatpata Chatwala,U0204,500,3,"KR Puram, Bangalore","Fast Food, Pizza, Street Food, Beverages, Shake"
3,R7506,Meghana Foods,U0300,300,2,"Sahakara Nagar, Bangalore","Biryani, Andhra, North Indian, Seafood"
4,R3521,Mealful Rolls - Meals In A Roll,U0479,400,5,"Jayanagar, Bangalore","Rolls, Fast Food"


In [3]:
restaurants_df.head()

,Name,URL,Cuisines,Area,Timing,Full_Address,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,Dinner Ratings,Dinner Reviews,Delivery Ratings,Delivery Reviews,KnownFor,PopularDishes,PeopleKnownFor,AverageCost,rest_id
0,Sri Udupi Park,https://www.zomato.com/bangalore/sri-udupi-par...,"South Indian, North Indian, Chinese, Street Fo...","Indiranagar, Bangalore",7am – 11pm (Today),"273, Monalisa, 6th Main, 100 Feet Road, Indira...",+919945977774,1,1,1,1,4.0,462,4.1,16000,NaN,"Filtered Coffee, Sambhar, Pav Bhaji, Gobi Manc...","Economical, Prompt Service, Hygiene, Quality F...",450,R0001
1,Meghana Foods,https://www.zomato.com/bangalore/meghana-foods...,"Biryani, Andhra, North Indian, Seafood","Indiranagar, Bangalore",Opens at 6:30pm,"544, First Floor, CMH Road, Near Indiranagar M...",+918041135050,1,1,1,0,4.3,1654,4.3,28600,Spicy Chicken Biryani,"Authentic Hyderabadi Biryani, Paneer Biryani, ...","Boneless Chicken Biryani, Ample Seating Area, ...",700,R0002
2,Donne Biriyani House,https://www.zomato.com/bangalore/donne-biriyan...,Biryani,"Indiranagar, Bangalore",11am – 11pm (Today),"8/ 9, 17th F Cross, 2nd Stage, Indiranagar, Ba...",+918861564169,1,1,1,0,3.9,411,3.5,33200,NaN,NaN,"Great Recommendations, Nice Taste, Great Ambia...",300,R0003
3,Domino's Pizza,https://www.zomato.com/bangalore/dominos-pizza...,"Pizza, Fast Food, Desserts","Indiranagar, Bangalore",10:57am – 12midnight (Today),"308, 2nd Stage, 100 Feet Road, Indiranagar, Ba...",+919916465787,1,1,1,0,2.4,422,4.4,8205,NaN,"Barbeque Chicken Pizza, Choco Lava Cake, White...","Value for Money, Packaging, Staff, Ambience, Food",400,R0004
4,KFC,https://www.zomato.com/bangalore/kfc-indiranagar,"Burger, Fast Food, Biryani, Desserts, Beverages","Indiranagar, Bangalore",11am – 11pm (Today),"38/1A, CMH Road, Indiranagar, Bangalore",+919513700040,1,1,1,0,2.8,673,4.0,9148,NaN,"Fiery Chicken, Chicken Popcorn, Rice Bowl, Wings","Elegantly Decorated, Great Recommendations, Vi...",400,R0005


In [4]:
def get_previous_ratings(user_id, usersorder_df):
    # Filter restaurants rated by the user
    user_ratings = usersorder_df[usersorder_df['user_id'] == user_id]
    
    # Sort by rating in descending order
    user_rated_restaurants = user_ratings.sort_values(by='rating', ascending=False)
    
    return user_rated_restaurants

# Example usage: Show previously rated restaurants for user_id 'U0001'
user_id = 'U0350'
previous_ratings = get_previous_ratings(user_id, usersorder_df)

# Display results
print(f"Previously rated restaurants by user {user_id}:")
for _, row in previous_ratings.iterrows():
    print(f"Restaurant name: {row['Name']}, Rating: {row['rating']},Price:{row['cost']},Cuisines:{row['Cuisines']}")


Previously rated restaurants by user U0350:
Restaurant name: Indian Food Court, Rating: 4,Price:1000,Cuisines:North Indian, Chinese, Fast Food
Restaurant name: A1 Rich Bowl, Rating: 3,Price:800,Cuisines:Mughlai
Restaurant name: Chai Point, Rating: 3,Price:300,Cuisines:Tea, North Indian, Fast Food, Street Food, Beverages
Restaurant name: Sardarji Londonwaley, Rating: 2,Price:900,Cuisines:North Indian, Chinese
Restaurant name: Cupcake Noggins, Rating: 2,Price:800,Cuisines:Bakery, Desserts


In [5]:
# Merge the two dataframes on 'rest_id'
merged_df = pd.merge(usersorder_df, restaurants_df, on='rest_id')

# Create a user-item matrix (users vs restaurants)
user_rest_matrix = merged_df.pivot_table(index='user_id', columns='rest_id', values='rating', fill_value=0)

# Apply Singular Value Decomposition (SVD) to decompose the matrix
svd = TruncatedSVD(n_components=20, random_state=42)  # Use 20 latent features
matrix_svd = svd.fit_transform(user_rest_matrix)

# Reconstruct the matrix to get predicted ratings
matrix_svd_reconstructed = np.dot(matrix_svd, svd.components_)

# Function to recommend top N restaurants for a specific user
def recommend_restaurants(user_id, num_recommendations=5):
    # Ensure user_id is a string and map it to the corresponding row in the user_rest_matrix
    if user_id not in user_rest_matrix.index:
        raise ValueError(f"User ID {user_id} not found in the user-item matrix.")
    
    # Get the index of the user_id
    user_idx = user_rest_matrix.index.get_loc(user_id)
    
    # Predict ratings for all restaurants for this user
    predicted_ratings = matrix_svd_reconstructed[user_idx]
    
    # Sort the restaurants by predicted rating in descending order
    recommended_restaurants_idx = np.argsort(predicted_ratings)[::-1][:num_recommendations]
    
    # Get restaurant details from the restaurants_df
    recommended_restaurants = restaurants_df.iloc[recommended_restaurants_idx]
    
    # Now merge with the original 'usersorder_df' to get actual ratings for the recommended restaurants
    recommended_restaurants_with_rating = pd.merge(recommended_restaurants, 
                                                  usersorder_df[usersorder_df['user_id'] == user_id][['rest_id', 'rating']], 
                                                  on='rest_id', how='left')
    
    # Create a table with the recommended restaurants and their ratings
    recommendation_table = recommended_restaurants_with_rating[['Name', 'Cuisines', 'AverageCost']]
    
    # Return the table as output
    return recommendation_table

# Example usage: Get top 5 restaurant recommendations for user with user_id="U0350"
user_id = "U0350"
recommended_restaurants = recommend_restaurants(user_id, num_recommendations=5)

# Print the recommendation table
print(recommended_restaurants)

                 Name                                       Cuisines  \
0  Rapture Roti Curry                                   North Indian   
1      Kailash Parbat           North Indian, Street Food, Beverages   
2           Chai Days                           Fast Food, Beverages   
3          ShakeShack  Beverages, Shake, Waffle, Ice Cream, Desserts   
4   XO Belgian Waffle             Desserts, Beverages, Waffle, Shake   

   AverageCost  
0           50  
1          500  
2          300  
3          150  
4          400  
